In [ ]:
# 1. Imports
import os
from pathlib import Path

# 2. Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# 3. Auto-detect dataset path in Drive
possible_paths = [
    "/content/drive/MyDrive/data/fgvc-aircraft-2013b",
    "/content/drive/MyDrive/fgvc-aircraft-2013b",
    "/content/drive/MyDrive/datasets/fgvc-aircraft-2013b",
]

source_root = None
for p in possible_paths:
    if os.path.exists(p):
        source_root = p
        print(f"✅ Found dataset source at: {source_root}")
        break

if source_root is None:
    raise FileNotFoundError(
        "❌ Could not find 'fgvc-aircraft-2013b' in Google Drive.\n"
        "Make sure it exists in MyDrive/data/ or MyDrive/"
    )

# 4. Define train/test paths (NO COPYING)
train_dir = os.path.join(source_root, "train")
test_dir  = os.path.join(source_root, "test")

# 5. Sanity checks
assert os.path.exists(train_dir), "❌ train/ folder not found"
assert os.path.exists(test_dir),  "❌ test/ folder not found"

print("✅ Train path:", train_dir)
print("✅ Test path:", test_dir)


Mounted at /content/drive
✅ Found dataset source at: /content/drive/MyDrive/fgvc-aircraft-2013b
✅ Train path: /content/drive/MyDrive/fgvc-aircraft-2013b/train
✅ Test path: /content/drive/MyDrive/fgvc-aircraft-2013b/test


In [ ]:
# Verify class folders exist
train_classes = [d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))]
test_classes  = [d for d in os.listdir(test_dir) if os.path.isdir(os.path.join(test_dir, d))]

print(f"📁 Train classes: {len(train_classes)}")
print(f"📁 Test classes:  {len(test_classes)}")

print("Example train class:", train_classes[:3])

if len(train_classes) == 0:
    raise RuntimeError("❌ No class folders found in train directory.")


📁 Train classes: 100
📁 Test classes:  100
Example train class: ['707_320', '727_200', '737_200']


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
print("🚀 Using device:", device)

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
test_dataset  = datasets.ImageFolder(test_dir, transform=test_transforms)

train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True
)

print(f"✅ Train samples: {len(train_dataset)}")
print(f"✅ Test samples:  {len(test_dataset)}")


🚀 Using device: cpu
✅ Train samples: 3334
✅ Test samples:  3333


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# Grab one batch to confirm everything works
images, labels = next(iter(train_loader))
print("🧪 Batch image shape:", images.shape)
print("🧪 Batch labels shape:", labels.shape)

images = images.to(device)
labels = labels.to(device)

print("✅ Batch successfully moved to GPU")


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


🧪 Batch image shape: torch.Size([64, 3, 224, 224])
🧪 Batch labels shape: torch.Size([64])
✅ Batch successfully moved to GPU


In [ ]:
import torch
import torchvision
from torch import nn
from timeit import default_timer as timer

# 1. Re-create the model (EfficientNet B0)
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
model = torchvision.models.efficientnet_b0(weights=weights)

# 2. FREEZE layers first (Feature Extraction)
for param in model.features.parameters():
    param.requires_grad = False

# 3. Add the classifier head (with Dropout for regularization)
torch.manual_seed(42)
model.classifier = nn.Sequential(
    nn.Dropout(p=0.3), # Increased dropout to prevent overfitting
    nn.Linear(in_features=1280, out_features=100) # 100 classes
)

# 4. UNFREEZE the last 20% of the base model (The "Fine-Tuning" Magic)
# This lets the model learn specific aircraft features like wing shapes
for param in model.features[-3:].parameters():
    param.requires_grad = True

# 5. Setup for Training
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Use a much lower learning rate for fine-tuning
loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1) # Helps with hard classes
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001) # 10x smaller learning rate

# 6. Train!
print("Starting Fine-Tuning...")
epochs = 15  # Needs more time to adjust
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}\n-------------------------------")
    model.train()
    train_loss, train_acc = 0, 0

    # Training Loop
    for batch, (X, y) in enumerate(train_loader):
        X, y = X.to(device), y.to(device)
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= len(train_loader)
    train_acc /= len(train_loader)
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")

# 7. Save the Improved Model
print("Saving improved model...")
torch.save(model.state_dict(), "aircraft_model.pth")
from google.colab import files
files.download("aircraft_model.pth")

Starting Fine-Tuning...
Epoch 1/15
-------------------------------


KeyboardInterrupt: 